<a href="https://colab.research.google.com/github/nerdk312/RL_implements/blob/master/Actor_critic_pytorch_separate_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [0]:
env = gym.make('CartPole-v0')
gamma = 0.99
log_interval = 10

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
#SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

class Policy(nn.Module):
    '''
    Implements actor
    '''
    def __init__(self):
        super(Policy,self).__init__()
        self.affine1 = nn.Linear(4,128)

        # actor's layer
        self.action_head = nn.Linear(128,2)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self,x):
        '''
        forward of both actor
        '''
        x =  F.relu(self.affine1(x))

        # actor: choose action to take from state s_t by returning the probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # return values for both actor and critic as a tuple of 2 values:
        #1. a list with the probability of each ation over the action space
        return action_prob

In [0]:
class Value(nn.Module):
    '''
    Implement critic
    '''
    def __init__(self):
        super(Value,self).__init__()
        
        self.affine1 = nn.Linear(4,128)
        self.value_head = nn.Linear(128,1)
        
        self.state_list = []

    def forward(self,x):
        x =  F.relu(self.affine1(x))
        state_values =  self.value_head(x)

        return state_values

In [0]:
policy_model = Policy()
policy_optimizer = optim.Adam(policy_model.parameters(), lr = 3e-3)
value_model = Value()
value_optimizer = optim.Adam(value_model.parameters(), lr = 3e-3)
eps = np.finfo(np.float32).eps.item()

In [0]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs = policy_model(state)
    state_value = value_model(state)

    # create a categorical distribution over the list of probabilies of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    policy_model.saved_actions.append(m.log_prob(action))
    value_model.state_list.append(state_value)

    #model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()

def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop
    """
    R = 0
    saved_actions = policy_model.saved_actions
    state_values = value_model.state_list
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic ( value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in policy_model.rewards[::-1]:
        # Calculate the discounted value
        R = r + gamma*R
        returns.insert(0,R) # Nawid - This gets the return for each time step

    returns = torch.tensor(returns)
    returns = (returns -  returns.mean())/ (returns.std() + eps) # Nawid -  Normalise the reward it seems

    for log_prob, value, R in zip(saved_actions,state_values, returns):
        advantage = R - value.item() # Nawid - Value function is used as the baseline and in this case the Q value is the actual return

        # Calculate actor (policy) loss for each time step
        policy_losses.append(-log_prob*advantage)
        
        # Calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R]))) # Nawid-  Difference between the predicted value and the actual value


    # sum up all the values of policy losses and value losses
    policy_loss =  torch.stack(policy_losses).sum() 
    # Reset gradients
    policy_optimizer.zero_grad()
    
    # Perform backprop
    policy_loss.backward()
    policy_optimizer.step()


    value_loss = torch.stack(value_losses).sum()
    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()

    # reset rewards and action buffer
    del policy_model.rewards[:]
    del policy_model.saved_actions[:]
    del value_model.state_list[:]
    

In [0]:
def main():
    running_reward = 10

    # run infinitely many_episodes
    for i_episode in count(1):

        # reset environment and epsiode reward
        state = env.reset()
        ep_reward = 0 

        # for each episode, only run 9999 steps so we dont infinite loop while learning
        for t in range(1,10000):

            # select action from policy
            action =  select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            policy_model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        
        # Update cumulative reward
        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward

        # perform backprob
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        
        # check if we have 'solved the cartpole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break
        

main()

Episode 10	Last reward: 23.00	Average reward: 16.12
Episode 20	Last reward: 32.00	Average reward: 21.79
Episode 30	Last reward: 36.00	Average reward: 28.46
Episode 40	Last reward: 14.00	Average reward: 36.26
Episode 50	Last reward: 48.00	Average reward: 42.66
Episode 60	Last reward: 51.00	Average reward: 42.62
Episode 70	Last reward: 43.00	Average reward: 49.72
Episode 80	Last reward: 76.00	Average reward: 53.45
Episode 90	Last reward: 84.00	Average reward: 57.50
Episode 100	Last reward: 76.00	Average reward: 65.00
Episode 110	Last reward: 78.00	Average reward: 81.50
Episode 120	Last reward: 97.00	Average reward: 86.99
Episode 130	Last reward: 137.00	Average reward: 111.90
Episode 140	Last reward: 185.00	Average reward: 125.12
Episode 150	Last reward: 185.00	Average reward: 142.30
Episode 160	Last reward: 132.00	Average reward: 134.58
Episode 170	Last reward: 200.00	Average reward: 145.66
Episode 180	Last reward: 180.00	Average reward: 155.76
Episode 190	Last reward: 200.00	Average rew